In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from astropy.io import fits
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from tqdm import tqdm
import time
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import convolve, Gaussian1DKernel

In [ ]:
base_url = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/sv3/dark/"

# Obtengo las carpetas y archivos FITS desde la página HTML
def get_folders_and_fits(url):
    response = requests.get(url)
    # Verifica si la respuesta falló y muestra un mensaje
    if response.status_code != 200:
        print("Verifica tu conexión a Internet para asegurarte de que esté funcionando correctamente.")
        return [], []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    folders = []
    fits_files = []

    # Busco todos los enlaces en la página
    for link in soup.find_all('a', href=True):
        href = link['href']
        text = link.text
        if href.endswith('/') and not text.startswith('..') and not href.endswith('logs/'):
            # Es una carpeta
            folder_name = text.rstrip('/')
            folders.append(folder_name)
        elif text.startswith('redrock-sv3-dark-') or text.startswith('coadd-sv3-dark-'):
            # Es un archivo FITS
            fits_files.append(text)
    
    return folders, fits_files

# Explorar los archivos de la ultima, tercera, capa. Aqui estan los .fits
def explore_third_layer_folders(url):
    _ , fits_files = get_folders_and_fits(url)
    nstype=[]
    # FITS coadd y redrock
    for fits_file in tqdm(fits_files):
        with open('urlspectra.txt', 'a') as archivo:
            if fits_file.startswith('redrock-sv3-dark-'): 
                red_rock_fits_urls = url + fits_file  #aqui obtengo todos los urls que me llevan a todos los fits de redrock
                archivo.write(red_rock_fits_urls+"\n")
            if fits_file.startswith('coadd-sv3-dark-'): 
                coadd_fits_urls = url + fits_file
                archivo.write(coadd_fits_urls+"\n")
        print("copiado")
            

    
# Explorar las carpetas de la segunda capa 10016/ 10145/ etc/
def explore_second_layer_folders(url):
    folders, _ = get_folders_and_fits(url)
    for folder in folders:
        folder_url = url + folder + '/'
        explore_third_layer_folders(folder_url)                
                
# Explora las carpetas de la primera capa: 100/ 101/ etc/
def explore_first_layer_folders(url):
    folders, _ = get_folders_and_fits(url)
    for folder in folders:
        folder_url = url + folder + '/'
        explore_second_layer_folders(folder_url)
        
        
# Comienzo la exploración desde la URL base en la primera capa
explore_first_layer_folders(base_url)

In [ ]:
with open('urlspectra.txt', 'r') as archivo:
    lineas = archivo.readlines()  # Lee todas las líneas del archivo y las almacena en una lista
    for i in range(0, 7, 2): #Este rango va de el primero al tercer fits es decir los primeros 6 fits con 3 readrock y 3 coadd
        # Imprime dos líneas a la vez (par de líneas)
        if i + 1 < len(lineas):
            coadd=fits.open(lineas[i])
            readrock=fits.open(lineas[i + 1])
            read_readrock=Table.read(readrock, hdu=1)
            read_coadd = Table.read(coadd, hdu=1) 
            targetID_coadd=read_coadd['TARGETID'].data #Target ID lista
            Z=read_readrock['Z'].data                  # Redshift lista
            spectype=read_readrock['SPECTYPE'].data    # Tipo espectral lista
            #obtenemos los datos de flujo de todos los espectros para todos los fits de coadd
            Bflux_ns=coadd[4].data
            Rflux_ns=coadd[9].data
            Zflux_ns=coadd[14].data
            Cflux=[]
            spectracomplete=[]
            Bflux=[]
            Rflux=[]
            Zflux=[]
            for j in range(len(Bflux_ns)): # Estandarización de galaxias y QSO 
                spectracomplete = np.concatenate((Bflux_ns[j], Rflux_ns[j], Zflux_ns[j]))
                datos=spectracomplete
                # Paso 1: Calcular la media
                media = np.mean(datos)
                # Paso 2: Calcular la desviación estándar
                desviacion_estandar = np.std(datos) 
                # Paso 3: Aplicar la estandarización
                if desviacion_estandar!=0.0:
                    datos_estandarizados = (datos - media) / desviacion_estandar
                    Cflux.append(datos_estandarizados)
            for m in range(len(Cflux)):  #divide las listas ya estandarizadas por filtros de nuevo 
                lista_concatenada=Cflux[m]
                longitud_Bflux_ns = len(Bflux_ns[m])
                longitud_Rflux_ns = len(Rflux_ns[m])
                Bsflux = lista_concatenada[:longitud_Bflux_ns]
                Rsflux= lista_concatenada[longitud_Bflux_ns:longitud_Bflux_ns + longitud_Rflux_ns]
                Zsflux = lista_concatenada[longitud_Bflux_ns + longitud_Rflux_ns:]
                Bflux.append(Bsflux)
                Rflux.append(Rsflux)
                Zflux.append(Zsflux)
                
                
#Faltan las estrellas?
                
#Creación del archivo fits no se modifico nada 


            #Ahora, concatenamos todos los flujos (B,V,R) para cada espectro de todos los fits de coadd
            Lista = []
            #Datos completos de flujo lista de listas
            lista_cb = [','.join(map(str, arr)) for arr in Bflux]
            lista_cr = [','.join(map(str, arr)) for arr in Rflux]
            lista_cz = [','.join(map(str, arr)) for arr in Zflux]
            # Define los tipos de datos
            dtype = [('BFlux', 'S100000'), ('RFlux', 'S100000'),('ZFlux', 'S100000'), ('Spectype', 'S10'), ('Z', 'float'), ('Targetid', 'int')]
            # Crea un arreglo estructurado de NumPy
            data = np.zeros(len(lista_cb), dtype=dtype)
            data['BFlux'] = lista_cb
            data['RFlux'] = lista_cr
            data['ZFlux'] = lista_cz
            data['Spectype'] = spectype
            data['Z'] = Z
            data['Targetid'] = targetID_coadd
            nombre_archivo = 'DataDESI.fits'
            hdulist = fits.open(nombre_archivo, mode='append')
            # Crea una tabla FITS usando fits.BinTableHDU
            hdu = fits.BinTableHDU.from_columns(data)
            hdulist.append(hdu)
            hdulist.close()          
              

            
        

In [ ]:
from astropy.table import Table
espc = fits.open('DataDESI.fits')
Tabla_fit1 = Table.read(espc, hdu=1)
print (Tabla_fit1['RFlux'][-5])
#Tabla_fit2 = Table.read(espc, hdu=2)#De esta tabla podemos extraer los TARGET ID y validar el sepctype en otro catalogo
#print (len(Tabla_fit2['BFlux'][0]))